In [1]:
import time
from design import *
import importlib
import shutil
from utils import *
from openai import OpenAI
from prompts import *
import json
import numpy as np
from gymnasium.envs.robodesign.GPTWalker import GPTWalkerEnv
import os

In [ ]:
import prompts
class DGA:
    def __init__(self):
        api_key = "<api_key>"
        self.client = OpenAI(api_key=api_key)
        self.model = "gpt-4-turbo"
        
    def extract_code(self, text):
        match = re.search(r'```python\n(.*?)\n```', text, re.DOTALL)
        return match.group(1).strip() if match else None

    def indent_code(self, code):
        return "\n".join(line if line.strip() else line for line in code.split("\n"))

    def generate_rewardfunc(self, rewardfunc_nums, folder_name):

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content": rewardfunc_prompts + zeroshot_rewardfunc_format}
        ]

        responses = self.client.chat.completions.create(
            model=self.model, messages=messages, n=rewardfunc_nums
        )
        files = []
        for i, choice in enumerate(responses.choices):
            reward_code = self.extract_code(choice.message.content)
            if reward_code:
                full_code = self.indent_code(reward_code) + "\n"
                file_name =  f"GPTWalker_{i}.py"
                file_path = os.path.join(folder_name, "env", file_name)
                with open(file_path, "w") as fp:
                    fp.write(full_code)

                with open(file_path, "w") as fp:
                    fp.write(full_code)
                files.append(file_path)
                print(f"Saved: {file_path}")
        return files
    
    def generate_rewardfunc_div(self, rewardfunc_nums, folder_name):

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content": rewardfunc_prompts + zeroshot_rewardfunc_format}
        ]

        # 生成初始 Reward Function
        response = self.client.chat.completions.create(
            model=self.model, messages=messages, n=1, timeout=10
        )

        rewardfunc_files = []

        initial_code = self.extract_code(response.choices[0].message.content)
        if initial_code:
            reward_code = "import numpy as np\n" + self.indent_code(initial_code) + "\n"

            file_path = os.path.join(folder_name, "env", "GPTrewardfunc_0.py")
            with open(file_path, "w") as fp:
                fp.write(reward_code)
            rewardfunc_files.append(file_path)
            print(f"initial Saved: {file_path}")

        # 生成不同的多样化 Reward Functions
        for i in range(1, rewardfunc_nums):
            diverse_messages = messages + [
                {"role": "user", "content": rewardfunc_div_prompts + zeroshot_rewardfunc_format}
            ]

            response = self.client.chat.completions.create(
                model=self.model, messages=diverse_messages, n=1
            )

            diverse_code = self.extract_code(response.choices[0].message.content)
            if diverse_code:
                reward_code =  "import numpy as np\n" + self.indent_code(diverse_code) + "\n"
                file_path = os.path.join(folder_name, "env", f"GPTrewardfunc_{i}.py")
                with open(file_path, "w") as fp:
                    fp.write(reward_code)
                rewardfunc_files.append(file_path)
                print(f"Saved: {file_path}")

        return rewardfunc_files

    def generate_morphology(self, morphology_nums, folder_name):
        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_prompts + morphology_format}
        ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=morphology_nums                                                                                                                                                                                                                                                                                   
        )

        # 解析所有 response 里的参数
        for i, choice in enumerate(responses.choices):
            print(f"Response {i}:")
            print(json.dumps(choice.message.content, indent=4))

        parameter_list = [json.loads(choice.message.content).get('parameters', []) for choice in responses.choices]
        material_list = [compute_walker_volume(parameter) for parameter in parameter_list]

        xml_files = []
        for i, parameter in enumerate(parameter_list):
            if not isinstance(parameter, list):
                print(f"Skipping invalid parameter {i}: {parameter}")
                continue

            xml_file = walker_design(parameter)  
            filename = f"GPTWalker_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            xml_files.append(file_path)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            print(f"Successfully saved {filename}")
            
        return xml_files, material_list, parameter_list
    
    def generate_morphology_div(self, morphology_nums, folder_name):

        material_list = []
        xml_files = []
        parameter_list = []
        
        # 生成初始 morphology
        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_prompts + morphology_format}
        ]
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=1
        )
        

        initial_parameter = json.loads(response.choices[0].message.content)
        parameter_list.append(initial_parameter['parameters'])
        material_list.append(compute_walker_volume(initial_parameter['parameters']))
        messages.append({"role": "assistwalker", "content": json.dumps(initial_parameter)})

        logging.info(f"generate initial_parameter{initial_parameter['parameters']}" )

        xml_file = walker_design(initial_parameter['parameters'])  

        filename = f"GPTWalker_0.xml"
        file_path = os.path.join(folder_name, "assets", filename)
        with open(file_path, "w") as fp:
            fp.write(xml_file)

        xml_files.append(file_path)

        # 生成不同的多样化设计
        for i in range(1, morphology_nums):
            diverse_messages = messages + [
                {"role": "user", "content": morphology_div_prompts + morphology_format}
            ]
            
            response = self.client.chat.completions.create(
                model=self.model,
                messages=diverse_messages,
                response_format={'type': 'json_object'},
                n=1
            )

            diverse_parameter = json.loads(response.choices[0].message.content)
            material_list.append(compute_walker_volume(diverse_parameter['parameters'])) 
            parameter_list.append(diverse_parameter['parameters'])
            messages.append({"role": "assistwalker", "content": json.dumps(diverse_parameter)})
            logging.info(f"generate diverse_parameter{ diverse_parameter['parameters']}")
            xml_file = walker_design(diverse_parameter['parameters'])  
            filename = f"GPTWalker_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            xml_files.append(file_path)

        return xml_files, material_list, parameter_list


    def improve_rewardfunc(self, best_rewardfunc, rewardfunc_list, fitness_list, folder_name, step, rewardfunc_index, morphology_index):
        reward_improve_prompts = prompts.reward_improve_prompts

        for rewardfunc_file, fitness in zip(rewardfunc_list, fitness_list):
            with open(rewardfunc_file, "r") as fp:
                reward_content = fp.read()
            reward_improve_prompts += f"\nreward function:\n{reward_content}\nfitness: {fitness}\n"

        with open(best_rewardfunc, "r") as fp:
            best_reward_content = fp.read()
        reward_improve_prompts += f"\nbest reward function:\n{best_reward_content}\nbest fitness: {max(fitness_list)}\n"

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content":reward_improve_prompts+ zeroshot_rewardfunc_format}
        ]
        print(messages)
        response = self.client.chat.completions.create(
            model=self.model, messages=messages
        )

        print(response)
        reward_code = self.extract_code(response.choices[0].message.content)

        if reward_code:
            full_code = "import numpy as np \n" + self.indent_code(reward_code) + "\n"
            file_name =  f"GPTrewardfunc_refine_{step}_{rewardfunc_index}_{morphology_index}.py"
            file_path = os.path.join(folder_name, "env", file_name)
            with open(file_path, "w") as fp:
                fp.write(full_code)

        return file_path

    def improve_morphology(self, best_parameter, parameter_list, fitness_list, folder_name, rewardfunc_index, morphology_index, iteration):
        morphology_improve_prompts = prompts.morphology_improve_prompts
        for parameter_content, fitness in zip(parameter_list, fitness_list):
            morphology_improve_prompts = morphology_improve_prompts + f"parameter:{parameter_content} \n" + f"fintess:{fitness}"
        morphology_improve_prompts = morphology_improve_prompts + f"best parameter:{best_parameter} \n" + f"best fintess:{max(fitness_list)}" 

        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_improve_prompts + morphology_format}
        ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
        )
        # print(responses)
        parameter = json.loads(responses.choices[0].message.content).get('parameters', []) 
        print(parameter)
        xml_file = walker_design(parameter)  
        filename = f"GPTWalker_refine2_{rewardfunc_index}_{morphology_index}_{iteration}.xml"
        file_path = os.path.join(folder_name, "assets", filename)

        with open(file_path, "w") as fp:
            fp.write(xml_file)

        print(f"Successfully saved {filename}")
        return file_path, parameter


# Configuration

In [19]:

folder_name = "results/Div_m25_r5"
log_file = os.path.join(folder_name, "parameters_refine_only.log")
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")

# folder_name = setup_logging(div_flag=True)

best_fitness = float('-inf')  
best_morphology = None  
best_rewardfunc = None  
best_reward = None
best_material = None
best_efficiency = None

morphology_nums = 51
rewardfunc_nums = 11

fitness_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
efficiency_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
fitness_list = []
designer = DGA()


# print configuration info

In [4]:
logging.info(f"start!")

In [1]:
# Extracting all the parameters from the provided log and storing them in a list

rewardfunc_list

NameError: name 'rewardfunc_list' is not defined

# enter coarse optimization stage

In [15]:
morphology_list = [f'results/Div_m25_r5/assets/GPTWalker_{i}.xml' for i in range(0,51) ]
rewardfunc_list = [f'results/Div_m25_r5/env/GPTrewardfunc_{i}.py' for i in range(0,11)]

parameter_list =np.array([[ 1.  ,  0.5  ,  0.25 ,  0.05 ,  0.25 , -0.25 ,  0.015,  0.02 , 0.025,  0.01 ],[1.5, 1.0, 0.6, 0.2, 0.3, -0.4, 0.05, 0.04, 0.03, 0.02], [1.0, 0.7, 0.5, 0.2, 0.25, -0.15, 0.045, 0.035, 0.025, 0.015], [1.8, 1.2, 0.9, 0.3, 0.35, -0.25, 0.06, 0.055, 0.045, 0.025], [1.0, 0.55, 0.35, 0.1, 0.1, -0.2, 0.03, 0.06, 0.07, 0.04], [1.3, 0.9, 0.6, 0.2, 0.25, -0.2, 0.04, 0.05, 0.05, 0.03], [1.2, 0.6, 0.3, 0.15, 0.2, -0.05, 0.065, 0.045, 0.03, 0.025], [0.8, 0.5, 0.2, 0.05, 0.1, -0.15, 0.04, 0.02, 0.01, 0.025], [1.1, 0.85, 0.55, 0.25, 0.3, -0.25, 0.035, 0.07, 0.08, 0.04], [2.0, 1.0, 0.5, 0.2, 0.25, -0.3, 0.02, 0.03, 0.04, 0.035], [1.2, 0.9, 0.4, 0.1, 0.15, -0.05, 0.025, 0.035, 0.045, 0.02], [1.0, 0.45, 0.25, 0.05, -0.05, -0.3, 0.02, 0.025, 0.03, 0.015], [0.9, 0.6, 0.4, 0.2, 0.25, -0.2, 0.02, 0.04, 0.06, 0.03], [0.6, 0.3, 0.15, 0.05, 0.1, -0.1, 0.04, 0.05, 0.06, 0.02], [0.5, 0.35, 0.2, 0.05, 0.15, -0.15, 0.035, 0.06, 0.075, 0.04], [0.8, 0.5, 0.3, 0.1, 0.2, -0.3, 0.04, 0.08, 0.12, 0.06], [0.6, 0.45, 0.15, 0.05, 0.1, -0.2, 0.025, 0.04, 0.055, 0.03], [1.0, 0.2, 0.1, 0.05, 0.05, -0.4, 0.03, 0.07, 0.1, 0.05], [0.8, 0.55, 0.25, 0.1, -0.05, -0.2, 0.045, 0.1, 0.15, 0.06], [0.6, 0.4, 0.3, 0.1, 0.2, -0.25, 0.055, 0.04, 0.03, 0.025], [1.2, 0.8, 0.4, 0.2, -0.1, -0.25, 0.03, 0.04, 0.06, 0.02], [1.0, 0.6, 0.4, 0.1, 0.2, -0.1, 0.02, 0.04, 0.05, 0.03], [1.5, 1.0, 0.7, 0.3, 0.4, -0.2, 0.05, 0.06, 0.07, 0.04], [1.0, 0.6, 0.25, 0.1, 0.15, -0.25, 0.03, 0.035, 0.04, 0.02], [1.2, 0.6, 0.3, 0.1, 0.12, -0.08, 0.025, 0.045, 0.065, 0.05], [1.7, 0.9, 0.45, 0.15, 0.25, -0.3, 0.025, 0.05, 0.08, 0.035], [1.5, 1.1, 0.6, 0.2, 0.35, -0.25, 0.02, 0.03, 0.04, 0.02], [1.2, 0.3, 0.15, 0.05, 0.1, -0.15, 0.02, 0.07, 0.12, 0.06], [1.0, 0.8, 0.5, 0.2, 0.25, -0.1, 0.025, 0.05, 0.1, 0.05], [1.0, 0.7, 0.4, 0.1, 0.2, -0.2, 0.03, 0.03, 0.06, 0.04], [0.9, 0.5, 0.25, 0.05, 0.1, -0.5, 0.03, 0.035, 0.07, 0.065], [1.8, 1.3, 0.6, 0.15, 0.25, -0.2, 0.025, 0.07, 0.12, 0.05], [1.0, 0.7, 0.5, 0.1, 0.2, -0.1, 0.01, 0.04, 0.08, 0.02], [1.0, 0.9, 0.6, 0.1, 0.2, -0.2, 0.02, 0.03, 0.05, 0.025], [1.2, 0.6, 0.3, 0.1, 0.25, -0.2, 0.02, 0.03, 0.08, 0.05], [0.8, 0.5, 0.3, 0.1, 0.15, -0.15, 0.015, 0.025, 0.035, 0.02], [1.0, 0.6, 0.35, 0.05, 0.1, -0.3, 0.02, 0.05, 0.1, 0.06], [1.0, 0.7, 0.3, 0.1, 0.15, -0.2, 0.025, 0.06, 0.12, 0.04], [1.2, 0.4, 0.2, 0.1, 0.2, -0.6, 0.02, 0.07, 0.15, 0.045], [0.8, 0.4, 0.2, 0.05, 0.1, -0.3, 0.015, 0.04, 0.1, 0.06], [0.7, 0.2, 0.05, 0.01, 0.2, -0.3, 0.01, 0.03, 0.08, 0.06], [1.0, 0.3, 0.15, 0.05, 0.1, -0.2, 0.02, 0.08, 0.15, 0.025], [1.5, 0.9, 0.6, 0.2, 0.25, -0.15, 0.018, 0.02, 0.025, 0.015], [0.85, 0.5, 0.25, 0.05, 0.1, -0.25, 0.015, 0.035, 0.05, 0.03], [1.0, 0.8, 0.5, 0.15, 0.2, -0.25, 0.02, 0.03, 0.035, 0.015], [1.0, 0.5, 0.3, 0.15, 0.3, -0.25, 0.04, 0.03, 0.02, 0.015], [1.2, 0.8, 0.4, 0.15, 0.3, -0.4, 0.02, 0.03, 0.04, 0.035], [0.9, 0.6, 0.25, 0.05, 0.1, -0.2, 0.02, 0.01, 0.03, 0.015], [1.0, 0.5, 0.25, 0.05, 0.25, -0.25, 0.015, 0.02, 0.025, 0.01], [1.4, 0.9, 0.6, 0.2, 0.3, -0.15, 0.025, 0.015, 0.015, 0.01], [1.45, 1.05, 0.6, 0.05, 0.1, 0, 0.2, 0.05,0.04,0.06]])

material_list = [compute_walker_volume(parameter) for parameter in parameter_list]


In [16]:
parameter_list.shape

(51, 10)

In [21]:
for i, rewardfunc in enumerate(rewardfunc_list):
    for j, morphology in enumerate(morphology_list):
        # if i not in [10]:
        #     continue
        if j not in [50] or i not in [10]:
            continue

        print(i, rewardfunc)
        print(j, morphology)
        shutil.copy(morphology, "GPTWalker.xml")
        shutil.copy(rewardfunc, "GPTrewardfunc.py")         

        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTWalkerEnv._get_rew = _get_rew

        # model_path = Train(j,  i, folder_name, total_timesteps=5e5)
        model_path = "results/Div_m25_r5/coarse/SAC_morphology50_rewardfunc0_500000.0steps"
        fitness, reward = Eva(model_path)
        material = material_list[j]
        efficiency = fitness/material
        fitness_matrix[i][j] = fitness
        efficiency_matrix[i][j] = efficiency
        
        logging.info("___________________finish coarse optimization_____________________")
        logging.info(f"morphology: {j}, rewardfunc: {i}, material cost: {material} reward: {reward} fitness: {fitness} efficiency: {efficiency}")

        if fitness > best_fitness:
            best_fitness = fitness
            best_morphology = morphology
            best_efficiency = efficiency
            best_rewardfunc = rewardfunc
            best_material = material

10 results/Div_m25_r5/env/GPTrewardfunc_10.py
50 results/Div_m25_r5/assets/GPTWalker_50.xml
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [22]:
efficiency_matrix_select = efficiency_matrix 
efficiency_matrix_select

array([[None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        No

# print coarse optimization info

# configuration of fine optimization

# enter fine optimization stage

In [23]:
final_optimized_results = []  # 用来记录每个 coarse_best 的最优结果
coarse_best = [(10,50)]
for rewardfunc_index, morphology_index in coarse_best:
    
    morphology = morphology_list[morphology_index]
    parameter = parameter_list[morphology_index]
    rewardfunc = rewardfunc_list[rewardfunc_index]
    
    best_efficiency = efficiency_matrix_select[rewardfunc_index][morphology_index]
    best_fitness = fitness_matrix[rewardfunc_index][morphology_index]
    best_morphology = morphology
    best_parameter = parameter
    best_rewardfunc = rewardfunc
    best_material = compute_walker_volume(parameter)
    
    
    logging.info(f"Initial morphology:{morphology}")
    logging.info(f"Initial parameter:{parameter}" )
    logging.info(f"Initial rewardfunc:{rewardfunc}" )
    logging.info(f"Initial fitness:{best_fitness}" )
    logging.info(f"Initial efficiency:{best_efficiency}" )
    iteration = 0
    print(f"Initial parameter:{parameter}")
    while True:
        improved = False  # 标记是否有改进，方便控制循环

        designer = DGA()
        iteration +=1   
         # -------- 优化 morphology --------
        improved_morphology, improved_parameter = designer.improve_morphology(
            best_parameter,
            parameter_list[morphology_index],  # 这本身已经是list结构，可以保留
            [efficiency_matrix_select[rewardfunc_index, morphology_index]],  # 👈 用 [] 包装成列表
            folder_name,
            rewardfunc_index, 
            morphology_index,
            iteration
        )
        
        print("improved parameter", improved_parameter)
        shutil.copy(improved_morphology, "GPTWalker.xml")
        shutil.copy(best_rewardfunc, "GPTrewardfunc.py")
        
        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTWalkerEnv._get_rew = _get_rew
        try:
            model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=5e5)
            improved_fitness, _ = Eva(model_path)
        except Exception as e:
            print(f"Error evaluating design: {e}")
            continue
        improved_material = compute_walker_volume(improved_parameter)
        improved_efficiency = improved_fitness / improved_material

        if improved_efficiency > best_efficiency:

            best_fitness = improved_fitness
            best_morphology = improved_morphology
            best_parameter = improved_parameter
            best_material = improved_material
            best_efficiency = improved_efficiency
            improved = True
            logging.info(f"Morphology optimization improved iteration {iteration}: material={improved_material}, fitness={improved_fitness}, efficiency={improved_efficiency}")

        # -------- 没有进一步改进，跳出循环 --------
        if not improved:
            logging.info("Not improved Morphology!")
            logging.info("____________________________________________")
            improved = False
            # break
            
        iteration +=1        
        # -------- 优化 reward function --------
        improved_rewardfunc = designer.improve_rewardfunc(
            best_rewardfunc,
            [rewardfunc_list[rewardfunc_index]],
            [efficiency_matrix_select[rewardfunc_index, morphology_index]],
            folder_name,
            rewardfunc_index, 
            morphology_index,
            iteration
        )
        
        shutil.copy(best_morphology, "GPTWalker.xml")
        shutil.copy(improved_rewardfunc, "GPTrewardfunc.py")
        
        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTWalkerEnv._get_rew = _get_rew
        try:
            model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=5e5)
            improved_fitness, _ = Eva(model_path)
            improved_material = compute_walker_volume(best_parameter)
            improved_efficiency = improved_fitness / improved_material
            print("improved_fitness", improved_fitness)
        except Exception as e:
            print(f"Error evaluating design: {e}")
            continue

        if improved_efficiency > best_efficiency:
            best_fitness = improved_fitness
            best_rewardfunc = improved_rewardfunc
            best_material = improved_material
            best_efficiency = improved_efficiency
            improved = True

            logging.info(f"Reward optimization improved iteration {iteration}: material={improved_material}, fitness={improved_fitness}, efficiency={improved_efficiency}")
        
        if not improved:
            logging.info("Not improved Reward!")
            logging.info("____________________________________________")
            break
            
    # 保存当前 coarse_best 的最终最优结果
    final_optimized_results.append({
        "best_morphology": best_morphology,
        "best_parameter": best_parameter,
        "best_rewardfunc": best_rewardfunc,
        "best_fitness": best_fitness,
        "best_material": best_material,
        "best_efficiency": best_efficiency,
        "best_iteration":iteration
    })
    logging.info("____________________________________________")
    logging.info(f"Final optimized result: rewardfunc_index{rewardfunc_index} morphology_index{morphology_index}")
    logging.info(f"  Morphology: {best_morphology}")
    logging.info(f"  Parameter: {best_parameter}")
    logging.info(f"  Rewardfunc: {best_rewardfunc}")
    logging.info(f"  Fitness: {best_fitness}")
    logging.info(f"  Material: {best_material}")
    logging.info(f"  Efficiency: {best_efficiency}")
    logging.info("____________________________________________")

Initial parameter:[1.45 1.05 0.6  0.05 0.1  0.   0.2  0.05 0.04 0.06]
[1.55, 1.05, 0.65, 0.07, 0.12, -0.1, 0.15, 0.045, 0.05, 0.07]
Successfully saved GPTWalker_refine2_10_50_1.xml
improved parameter [1.55, 1.05, 0.65, 0.07, 0.12, -0.1, 0.15, 0.045, 0.05, 0.07]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
[{'role': 'system', 'content': 'You are a reinforcement learning reward function designer'}, {'role': 'user', 'content': '\nYou are a reward engineer trying to write reward functions to solve refiforcement learning tasks as effective as possible.\nDescription: The walker is a two-dimensional bipedal robot consisting of seven main body parts - a single torso at the top (with the two legs splitting after the torso), two thighs in the middle below

In [ ]:

logging.info(f"{final_optimized_results}")

# logging.info(f"fine optimization end: best material cost: {best_material}  fitness: {improved_fitness} merterial_efficiency: {improved_material_efficiency}")

In [24]:
final_optimized_results

[{'best_morphology': 'results/Div_m25_r5/assets/GPTWalker_refine2_10_50_3.xml',
  'best_parameter': [1.6,
   1.1,
   0.7,
   0.08,
   0.13,
   -0.12,
   0.045,
   0.04,
   0.06,
   0.08],
  'best_rewardfunc': 'results/Div_m25_r5/env/GPTrewardfunc_refine_10_50_6.py',
  'best_fitness': 3.0380093859117197,
  'best_material': 0.03829601444725958,
  'best_efficiency': 79.32964904469625,
  'best_iteration': 8}]

2025-04-07 06:13:50,208 - Final optimized result: rewardfunc_index0 morphology_index0
2025-04-07 06:13:50,208 -   Morphology: results/Div_m25_r5/assets/GPTWalker_refine_0_0_0.xml
2025-04-07 06:13:50,208 -   Parameter: [1.05, 0.55, 0.3, 0.1, 0.3, -0.25, 0.015, 0.025, 0.03, 0.015]
2025-04-07 06:13:50,208 -   Rewardfunc: results/Div_m25_r5/env/GPTWalker_0_0_1.py
2025-04-07 06:13:50,208 -   Fitness: 6.466572426996648
2025-04-07 06:13:50,208 -   Material: 0.003643200280612963
2025-04-07 06:13:50,208 -   Efficiency: 1774.970336220071

In [ ]:
{'best_morphology': 'results/Div_m25_r5/assets/GPTWalker_11.xml',
  'best_parameter': array([ 1.   ,  0.45 ,  0.25 ,  0.05 , -0.05 , -0.3  ,  0.02 ,  0.025,
          0.03 ,  0.015]),
  'best_rewardfunc': 'results/Div_m25_r5/env/GPTrewardfunc_refine_1_11_1.py',
  'best_fitness': 14.883819809931813,
  'best_material': 0.0033798300964870192,
  'best_efficiency': 4403.718348269042,
  'best_iteration': 2}]

In [22]:
# Robodesign best

morphology = "results/Div_m25_r5/assets/GPTWalker_11.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_refine_1_11_1.py"

morphology_index=9999
rewardfunc_index=9999

parameter = [ 1.   ,  0.45 ,  0.25 ,  0.05 , -0.05 , -0.3  ,  0.02 ,  0.025,
          0.03 ,  0.015]


shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign best best 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign best 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Robodesign best 1e6 steps train

fitness:3.4504453808491076
efficiency:1020.8931462074042


In [ ]:
# Robodesign best

morphology = "results/Div_m25_r5/assets/GPTWalker_refine.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_3.py"



# morphology = "results/Div_m25_r5/assets/GPTWalker_0.xml"
# rewardfunc = "results/Div_m25_r5/env/GPTWalker_0_0_1.py"

morphology_index=9998
rewardfunc_index=9998


parameter = [0.95, 0.75, 0.45, 0.15, 0.2, -0.2, 0.01, 0.01, 0.01, 0.008]


shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign best best 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign best 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

In [34]:
parameter = [ 1.   ,  0.5  ,  0.25 ,  0.05 ,  0.25 , -0.25 ,  0.015,  0.02, 0.025,  0.01 ]
parameter = [1.1, 0.75, 0.45, 0.15, 0.2, -0.2, 0.015, 0.012, 0.012, 0.008]
material = compute_walker_volume(parameter)
material




0.0009984965818806487

In [ ]:
# Robodesign best

morphology = "results/Div_m25_r5/assets/GPTWalker_refine.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_0.py"



# morphology = "results/Div_m25_r5/assets/GPTWalker_0.xml"
# rewardfunc = "results/Div_m25_r5/env/GPTWalker_0_0_1.py"

morphology_index=999
rewardfunc_index=999

# parameter = [1.05, 0.55, 0.3, 0.1, 0.3, -0.25, 0.015, 0.025, 0.03, 0.015]
# parameter = [ 1.   ,  0.5  ,  0.25 ,  0.05 ,  0.25 , -0.25 ,  0.015,  0.02, 0.025,  0.01 ]

parameter = [0.95, 0.75, 0.45, 0.15, 0.2, -0.2, 0.01, 0.01, 0.01, 0.008]


shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign best best 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign best 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

2025-04-07 01:54:53,368 - Initial morphology:results/Div_m25_r5/assets/GPTWalker_11.xml
2025-04-07 01:54:53,368 - Initial parameter:[ 1.     0.45   0.25   0.05  -0.05  -0.3    0.02   0.025  0.03   0.015]
2025-04-07 01:54:53,368 - Initial rewardfunc:results/Div_m25_r5/env/GPTrewardfunc_3.py
2025-04-07 01:54:53,368 - Initial fitness:5.010931913179973
2025-04-07 01:54:53,368 - Initial efficiency:1482.5987609224244

In [58]:
# coarse best

morphology = "results/Div_m25_r5/assets/GPTWalker_11.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_3.py"

morphology_index=888
rewardfunc_index=888

# parameter = [ 1.    , 0.45  , 0.25 ,  0.05  ,-0.05  ,-0.3  ,  0.02  , 0.025  ,0.03  , 0.015]


morphology = "results/Div_m25_r5/assets/GPTWalker_0.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_3.py"

parameter =[ 1.   ,  0.5  ,  0.25 ,  0.05 ,  0.25 , -0.25 ,  0.015,  0.02 ,
        0.025,  0.01 ]

shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
# model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign coarse best best 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign coarse best 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Robodesign coarse best 1e6 steps train

fitness:9.459751803686697
efficiency:4109.826005402759


In [8]:
# human

morphology = "results/Div_m25_r5/assets/GPTWalker_50.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_10.py"

morphology_index=666
rewardfunc_index=666

parameter = [1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]

shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign human 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign human 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Robodesign human 1e6 steps train

fitness:4.777281062373332
efficiency:170.29255939736836


In [9]:
# Robodesign (w/o Morphology Design)

morphology = "results/Div_m25_r5/assets/GPTWalker_50.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_1.py"

morphology_index=555
rewardfunc_index=555

parameter = [1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]

shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign (w/o Morphology Design) 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign (w/o Morphology Design) 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Robodesign (w/o Morphology Design) 1e6 steps train

fitness:1.912675745085834
efficiency:68.17987965860266


In [19]:
parameter_list[0]

array([ 1.   ,  0.5  ,  0.25 ,  0.05 ,  0.25 , -0.25 ,  0.015,  0.02 ,
        0.025,  0.01 ])

In [60]:
# Robodesign (w/o Reward Design)

morphology = "results/Div_m25_r5/assets/GPTWalker_0.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_10.py"

morphology_index=444
rewardfunc_index=444

parameter =[ 1.   ,  0.5  ,  0.25 ,  0.05 ,  0.25 , -0.25 ,  0.015,  0.02 ,
        0.025,  0.01 ]
shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
# model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info(" Robodesign (w/o Reward Design) 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print(" Robodesign (w/o Reward Design) 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
 Robodesign (w/o Reward Design) 1e6 steps train

fitness:8.959412770453445
efficiency:3892.451764199232


In [12]:
# Robodesign (w/o diversity reflection)

morphology = "results/noDiv_m25_r5/assets/GPTWalker_20.xml"
rewardfunc = "results/noDiv_m25_r5/env/GPTrewardfunc_1.py"

morphology_index=333
rewardfunc_index=333

parameter = [ 0.6 ,   0.45,   0.3    ,0.15 ,  0.1 ,  -0.1  ,  0.04 ,  0.04   ,0.035 , 0.03 ]

shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info(" Robodesign  (w/o diversity reflection) 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print(" Robodesign (w/o diversity reflection) 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
 Robodesign (w/o diversity reflection) 1e6 steps train

fitness:5.337485313971282
efficiency:899.007501259998


In [13]:
# eureka reward
morphology = "results/eureka/assets/GPTWalker_0.xml"
rewardfunc = "results/eureka/env/GPTrewardfunc_2_1.py"

morphology_index=111
rewardfunc_index=111

parameter = [1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]

shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info("# eureka reward 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("# eureka reward 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
# eureka reward 1e6 steps train

fitness:3.919962300766995
efficiency:139.73228793182562


In [14]:
# eureka morphology

morphology = "results/eureka_morphology/assets/GPTWalker_5.xml"
rewardfunc = "results/eureka_morphology/env/GPTrewardfunc_0.py"

morphology_index=111
rewardfunc_index=111

parameter = [ 1.25   , 1.0 ,  0.75 ,   0.45  , 0.4 , -0.4  ,  0.025  , 0.018 ,  0.01 , 0.006 ]

shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info("eureka morphology 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("eureka morphology 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
eureka morphology 1e6 steps train

fitness:2.9077493339127254
efficiency:1946.5997575676536


In [ ]:
# Robodesign (w/o Reward Design w/o morphology design)

morphology = "results/Div_m25_r5/assets/GPTWalker_0.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_0.py"

morphology_index=123
rewardfunc_index=123

parameter = parameter_list[0]


shutil.copy(morphology, "GPTWalker.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTWalkerEnv._get_rew = _get_rew
        
# model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_1000000.0steps"
fitness, _ = Eva(model_path)
material = compute_walker_volume(parameter)
efficiency = fitness / material

logging.info(" (w/o Reward Design w/o morphology design) 1e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print(" (w/o Reward Design w/o morphology design) 1e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")